In [ ]:
from astropy.io import fits
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import pyarrow.parquet as parquet
import fast_histogram as fh
import time

In [ ]:
aspect = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet',filters=[('flags','=',0)]).to_pandas()


In [ ]:
metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet').to_pandas()


In [ ]:
boresight = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet').to_pandas()

In [ ]:
def get_second_time(group):
    if len(group) >= 2:
        return group.iloc[1]['time']
    else:
        return None  # If less than 2 entries, return None

boresight_second_time = boresight.groupby('eclipse').apply(get_second_time).reset_index()
boresight_second_time.columns = ['eclipse', 'second_boresight_time']

merged_df = pd.merge(merged, boresight_second_time, on='eclipse')

filtered_aspect = merged_df[(merged_df['time'] <= merged_df['second_boresight_time']) | merged_df['second_boresight_time'].isna()]

filtered_aspect = filtered_aspect.drop(columns=['second_boresight_time'])

In [ ]:
filtered_aspect

In [ ]:
dispersion = filtered_aspect.groupby(['eclipse']).agg({
    'ra': 'std',
    'dec': 'std',
    'time': 'count'}, skipna=False).reset_index()

In [ ]:
dispersion

In [ ]:
final = pd.merge(dispersion, metadata, on='eclipse', how='inner')


In [ ]:
final = final[(final['ra']<20) & (final['dec']<20)]

In [ ]:
plt.scatter(final['ra'],final['dec'],s=.1)
plt.ylabel('dec stdev')

In [ ]:
# get more than one leg 
multileg = merged_dispersion[(merged_dispersion['legs'] > 0)]


In [ ]:
outliers = merged_dispersion[(merged_dispersion['legs'] == 12) & (merged_dispersion['ra'] > 20)]
outliers

In [ ]:
merged_filtered = merged_dispersion[(merged_dispersion['legs'] == 0) & (merged_dispersion['ra'] < 10) & (merged_dispersion['dec'] < 10) & (merged_dispersion['obstype'] != 'unknown')]

In [ ]:
plt.scatter(merged_filtered['ra'],merged_filtered['dec'],s=.09)
plt.xlim(xmin=-.01,xmax=.04)
plt.ylim(ymin=-.01,ymax=.04)
plt.xlabel('ra std for one leg')
plt.ylabel('dec std for one leg')

In [ ]:
obstype = final.groupby(['obstype']).agg(
    {'ra' : ['median', 'std','mean'], 'dec' : ['median', 'std','mean']}, skipna=False).reset_index()

In [ ]:
plt.scatter(obstype['obstype'],obstype['ra']['mean'],marker='o',s=20, label='ra mean')
plt.scatter(obstype['obstype'],obstype['ra']['median'],marker='s',s=20, label='ra median')
plt.scatter(obstype['obstype'],obstype['dec']['mean'],marker='*',s=20, label= 'dec mean')
plt.scatter(obstype['obstype'],obstype['dec']['median'],marker='+',s=20,label='dec median')
plt.legend()

In [ ]:
plt.scatter(obstype['obstype'],obstype['ra']['std'])
plt.scatter(obstype['obstype'],obstype['dec']['std'])

In [ ]:
final[['eclipse', 'ra','dec','legs','obstype']].to_parquet('stdev_by_eclipse.parquet', index=False)


In [ ]:
# check which eclipses are missing 

merged_df = pd.merge(dispersion, final, on='eclipse', how='outer', indicator=True)

non_matching_in_df1 = merged_df[merged_df['_merge'] == 'left_only']

non_matching_in_df2 = merged_df[merged_df['_merge'] == 'right_only']

In [ ]:
non_matching_in_df1